# 빙산

소요시간: 100분

작성일시: 24.03.22 13:49:33

출처: https://www.acmicpc.net/problem/17144

### 접근 방법
1st
- 문제 조건에 충실하게 구현.
- python3 시간초과, pypy3 통과.

2nd
- 미세먼지 확산 시 인접 칸의 변화량을 hash에서 2차원 배열에 저장하는 것으로 변경
- 현재 칸의 미세먼지 감소는 기존 board에 바로 반영해도 상관없음
- python3 통과

### 시간 복잡도 분석
연산 횟수가 1000만 정도 됨.

### 새로 알게 된 것
딕셔너리가 항상 O(1)의 데이터 수정 및 탐색을 보장하진 않는다.  
해시 충돌이 발생하면 linear probing 기법을 사용한다.  
해시값에 이미 할당되어있으면 그 값을 다시 해싱하여 해시값을 산출하고, 비어있으면 그곳에 저장한다.  
읽기 연산도 동일하다. 따라서 해시 충돌이 발생하면 O(1)이 보장되지 않는다.  

### 주의할 점
.

### 기타 코멘트
2nd에 딕셔너리에 변화량을 저장했다가 2차원 배열에 저장하는 것으로 바꿔서 pypy3기준 1500ms 이상 줄였다.  
이렇게까지 큰 차이가 날 정도로 해시 충돌이 잦은가?? 조사해볼 가치가 있는듯.  
딕셔너리의 길이가 길어질 정도로 값이 많아지면 배열을 사용하는 것을 고려해보자.


In [ ]:
from collections import defaultdict
from sys import stdin

def make_path():
    a1, a2 = air
    upper = []
    for i in range(a1-1, -1, -1):
        upper.append((i, 0))
    for i in range(1, M):
        upper.append((0, i))
    for i in range(1, a1+1):
        upper.append((i, M-1))
    for i in range(M-2, 0, -1):
        upper.append((a1, i))
    lower = []
    for i in range(a2+1, N):
        lower.append((i, 0))
    for i in range(1, M):
        lower.append((N-1, i))
    for i in range(N-2, a2-1, -1):
        lower.append((i, M-1))
    for i in range(M-2, 0, -1):
        lower.append((a2, i))
    return upper, lower

def blow():
    for arr in [up_path, down_path]:
        for i in range(1, len(arr)):
            board[arr[i-1][0]][arr[i-1][1]] = board[arr[i][0]][arr[i][1]]
        board[arr[-1][0]][arr[-1][1]] = 0

def add_dust():
    cnt = 0
    for i in range(N):
        for j in range(M):
            if board[i][j] > 0:
                cnt += board[i][j]
    return cnt

def is_grid_in(r, c):
    return 0 <= r < N and 0 <= c < M

input = stdin.readline
RDLU = [(0, 1), (1, 0), (0, -1), (-1, 0)]
N, M, T = map(int, input().split())
board = []
air = []
dust_set = set()
for i in range(N):
    row = list(map(int, input().rstrip().split()))
    board.append(row)
    if row[0] == -1:
        air.append(i)
up_path, down_path = make_path()

for _ in range(T):
    changed = defaultdict(int)
    for i in range(N):
        for j in range(M):
            if board[i][j] >= 5:
                cnt = 0
                div5 = board[i][j] // 5
                for dx, dy in RDLU:
                    nx, ny = i + dx, j + dy
                    if is_grid_in(nx, ny) and board[nx][ny] != -1:
                        cnt += 1
                        changed[(nx, ny)] += div5
                changed[(i, j)] -= div5 * cnt
    for x, y in changed:
        board[x][y] += changed[(x, y)]
    blow()
print(add_dust())

In [ ]:
from sys import stdin

def diffuse(board):
    changed = [[0]*M for _ in range(N)]
    for i in range(N):
        for j in range(M):
            if board[i][j] >= 5:
                div5 = board[i][j] // 5
                for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                    nx, ny = i + dx, j + dy
                    if is_grid_in(nx, ny) and board[nx][ny] != -1:
                        changed[nx][ny] += div5
                        board[i][j] -= div5
    for i in range(N):
        for j in range(M):
            board[i][j] += changed[i][j]

def blow(board, air):
    a1, a2 = air, air+1
    for i in range(a1-2, -1, -1):
        board[i+1][0] = board[i][0]
    for i in range(1, M):
        board[0][i-1] = board[0][i]
    for i in range(1, a1+1):
        board[i-1][M-1] = board[i][M-1]
    for i in range(M-2, 0, -1):
        board[a1][i+1] = board[a1][i]
    board[a1][1] = 0

    for i in range(a2+2, N):
        board[i-1][0] = board[i][0]
    for i in range(1, M):
        board[N-1][i-1] = board[N-1][i]
    for i in range(N-2, a2-1, -1):
        board[i+1][M-1] = board[i][M-1]
    for i in range(M-2, 0, -1):
        board[a2][i+1] = board[a2][i]
    board[a2][1] = 0

def add_dust(board):
    return 2 + sum(map(sum, board))

def is_grid_in(r, c):
    return 0 <= r < N and 0 <= c < M

input = stdin.readline
N, M, T = map(int, input().split())
board = [list(map(int, input().rstrip().split())) for _ in range(N)]
air = 0
for i in range(N):
    if board[i][0] == -1:
        air = i
        break

for _ in range(T):
    diffuse(board)
    blow(board, air)
print(add_dust(board))